### Install dependencies

In [1]:
from jupyter_dash import JupyterDash
import pandas as pd
import numpy as np
from plotly import graph_objects as go
import app as app_file
import setup as setup_file
import dash_core_components as dcc
import pickle
from Configuration import *

### Serve Jupyter Dash

In [ ]:
JupyterDash.infer_jupyter_proxy_config()

### Define your static parameters

In [4]:
setup_applet = setup_file.create_app()
setup_applet.run_server(mode='inline', port=8891)

### View your sample population

In [5]:
with open('configs.pkl', 'rb') as f:
    configs = pickle.load(f)

In [6]:
sample_pop = configs.enc.initialize()
print(sample_pop)

[{'gene': array([1, 5, 7, 9, 0, 4, 3, 8, 6, 2]), 'fitness': array([0])}
 {'gene': array([8, 3, 1, 5, 0, 6, 9, 2, 4, 7]), 'fitness': array([0])}
 {'gene': array([5, 7, 0, 4, 1, 6, 3, 2, 9, 8]), 'fitness': array([0])}
 {'gene': array([6, 2, 3, 4, 8, 7, 5, 9, 0, 1]), 'fitness': array([0])}
 {'gene': array([7, 1, 9, 5, 6, 4, 2, 8, 3, 0]), 'fitness': array([0])}]


### Define your evaluation function

Below is an example fitness evaluation function for the single-objective Traveling Salesman Problem (TSP)

```
def TSP(ind):
    dist = np.loadtxt('dist.txt')

    ind['fitness'][0] += dist[0][ind['gene'][0]]
    ind['fitness'][0] += np.array([dist[ind['gene'][i - 1]][ind['gene'][i]] for i in range(1, configs.gene_size)]).sum()
    ind['fitness'][0] += dist[ind['gene'][-1]][0]

    return ind
```

In [7]:
def TSP(ind):
    dist = np.loadtxt('dist.txt')

    ind['fitness'][0] += dist[0][ind['gene'][0]]
    ind['fitness'][0] += np.array([dist[ind['gene'][i - 1]][ind['gene'][i]] for i in range(1, configs.gene_size)]).sum()
    ind['fitness'][0] += dist[ind['gene'][-1]][0]

    return ind

sample_pop[0] = TSP(sample_pop[0])
print(sample_pop[0])

{'gene': array([1, 5, 7, 9, 0, 4, 3, 8, 6, 2]), 'fitness': array([4948])}


### Define a custom phenotypic representation

In [8]:
def network(ind):
    locs = np.loadtxt('loc.txt')  
    
    fig = go.Figure(data=[go.Scatter(x=[locs[0][ind['gene'][i]] for i in range(configs.gene_size)],
                                     y=[locs[1][ind['gene'][i]] for i in range(configs.gene_size)],
                                     mode='lines')])
    return fig

print(network(sample_pop[0]))

Figure({
    'data': [{'mode': 'lines',
              'type': 'scatter',
              'x': [28.0, 72.0, 97.0, 3.0, 48.0, 15.0, 58.0, 59.0, 84.0, 55.0],
              'y': [36.0, 29.0, 44.0, 87.0, 84.0, 88.0, 80.0, 89.0, 8.0, 30.0]}],
    'layout': {'template': '...'}
})


### Save your static configurations

If you are satisfied with your evaluation function and custom visualization, save it to your configurations

In [9]:
configs.eval, configs.vis = TSP, network

### Run the application

In [11]:
main_app = app_file.create_app(configs)
main_app.run_server(port=8891)

Dash app running on http://127.0.0.1:8891/


In [12]:
configs.pop_size

10